In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(max_tokens=1024, temperature=0.1)

In [5]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()
p.parse(" Hello, How ,are, you")

['Hello', 'How', 'are', 'you']

In [6]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. in lower case. Do Not Reply with anything else"),
    ("human", "{question}")
])


In [7]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "포켓몬이 뭐야?"
})

['피카츄', '파이리', '꼬부기', '이상해씨', '꼬마돌']

In [8]:
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, 
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])
chef_chain = chef_prompt | chat

veg_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it. Translate Korean"), 
    ("human", "{recipe}")
])
veg_chain = veg_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine": "Korean"
})

Great choice! How about trying to make Bibimbap? It's a delicious and colorful Korean rice dish that is packed with flavor. Here's a simple recipe for you to try at home:

Ingredients:
- 2 cups cooked white rice
- 1 carrot, julienned
- 1 zucchini, julienned
- 1 cup bean sprouts
- 1 cup spinach
- 4 shiitake mushrooms, sliced
- 1/2 pound beef (you can use ribeye or sirloin), thinly sliced
- 4 eggs
- Sesame oil
- Soy sauce
- Gochujang (Korean chili paste)
- Sesame seeds
- Salt and pepper
- Vegetable oil

Instructions:
1. Marinate the beef slices with 2 tablespoons of soy sauce, 1 tablespoon of sesame oil, and a pinch of black pepper. Set aside to marinate for at least 15 minutes.
2. Heat a pan over medium heat and add a little vegetable oil. Stir-fry the marinated beef until cooked through. Remove from the pan and set aside.
3. In the same pan, add a little more oil if needed and stir-fry the carrots, zucchini, and mushrooms separately until they are cooked but still slightly crunchy. Sea

AIMessageChunk(content='Ingredients:\n- 2 cups cooked white rice\n- 1 carrot, julienned (you can use a vegetable peeler to create thin strips)\n- 1 zucchini, julienned\n- 1 cup mung bean sprouts (you can blanch them in boiling water for about 1 minute)\n- 1 cup spinach (you can blanch and squeeze out excess water)\n- 4 shiitake mushrooms, sliced\n- Vegetarian beef substitute (you can use plant-based beef slices or tofu as a replacement), thinly sliced\n- 4 eggs (you can use a plant-based egg substitute)\n- Sesame oil\n- Soy sauce (use a vegetarian or vegan version)\n- Gochujang (Korean chili paste)\n- Sesame seeds\n- Salt and pepper\n- Vegetable oil\n\nInstructions:\n1. Marinate the vegetarian beef slices with 2 tablespoons of soy sauce, 1 tablespoon of sesame oil, and a pinch of black pepper. Set aside to marinate for at least 15 minutes.\n2. Heat a pan over medium heat and add a little vegetable oil. Stir-fry the marinated vegetarian beef until cooked through. Remove from the pan and

In [14]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(temperature=0.1, 
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat
chain.invoke({
    "country": "Turkey",
})

AI: 
        Here is what I know:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')